In [ ]:
import networkx as nx
import numpy as np
import random
import time

def dhicm_dynamic_p(G, k):
    S = set()
    V = set(G.nodes())  # Assuming G is a NetworkX graph or a similar graph representation
    ddv = {v: G.in_degree(v) for v in V}

    for i in range(k):
        u = max((v for v in V - S), key=lambda v: ddv[v])
        S.add(u)

        for v in G.predecessors(u):
            if v not in S:
              if u in G.predecessors(v):
                common_neighbors = set(G.predecessors(v)).intersection(set(G.predecessors(u)))
                p = 0.01 + ((G.in_degree(u) + G.in_degree(v))/G.number_of_nodes()) + (len(common_neighbors)/G.number_of_nodes())
                p *= G[u][v]['weight'] # accounting for edge weight
                ddv[v] = G.in_degree(v) - 1 - ((G.in_degree(u) - 1) * p)
    return S

def degree_centrality(G, k):
    S = set()
    V = set(G.nodes())  # Assuming G is a NetworkX graph or a similar graph representation
    ddv = {v: G.in_degree(v) for v in V}

    for i in range(k):
        u = max((v for v in V - S), key=lambda v: ddv[v])
        S.add(u)

    return S

def modified_ICM(graph_object,S,mc):
    """
    Inputs: graph_object: must be networkx directed graph
            S:  List of seed nodes
            p:  Disease propagation probability
            mc: Number of Monte-Carlo simulations,
    Output: Average number of nodes influenced by seed nodes in S
    """

    # Loop over the Monte-Carlo Simulations
    spread = []
    for i in range(mc):

        # Simulate propagation process
        new_active, A = S[:], S[:]
        while new_active:
            # 1. Find out-neighbors(i.e. successor nodes) for each newly active node
            targets = modified_propagate(graph_object, new_active)

            # 2. Determine newly activated neighbors (set seed and sort for consistency)
            np.random.seed(i)
            new_ones = []
            for (curr_target, p) in targets:
              if np.random.uniform(0,1) < p:
                new_ones.append(curr_target)

            # 3. Find newly activated nodes and add to the set of activated nodes
            new_active = list(set(new_ones) - set(A))
            A += new_active

        spread.append(len(A))

    return(np.mean(spread),A)

def modified_propagate(g, new_active):
    targets = []
    for node in new_active:
      for neighbor in g.predecessors(node):
        common_neighbors = set(G.predecessors(node)).intersection(set(G.predecessors(neighbor)))
        p = 0.01 + ((G.in_degree(node) + G.in_degree(neighbor))/G.number_of_nodes()) + (len(common_neighbors)/G.number_of_nodes())
        p *= G[neighbor][node]['weight'] # accounting for edge weight
        targets.append((neighbor, p))

    return targets

In [ ]:
# creating graph
file1 = open('higgs-retweet_network.edgelist', 'r')
Lines = file1.readlines()
edge_list = []
for line in Lines:
  u_v = line.strip().split()
  edge_list.append((int(u_v[0]), int(u_v[1]), int(u_v[2])))

G = nx.DiGraph()
G.add_weighted_edges_from(edge_list)  # using a list of edge tuples
total_edge_weight = G.size(weight = 'weight')

In [ ]:
total_edge_weight = 0
for curr_node in G.nodes():
  for pred in G.predecessors(curr_node):
    total_edge_weight += G[pred][curr_node]['weight']
total_edge_weight

354930

In [ ]:
#getting seed nodes using dhicm
start_time = time.time()
s_dhicm = dhicm_dynamic_p(G, 1000)
s_degree = degree_centrality(G, 1000)
print("Time getting seed set: ", time.time() - start_time)

Time getting seed set:  185.78967332839966


In [ ]:
start_time = time.time()
#getting mean spread
(mean_spread, A) = modified_ICM(G, list(s_dhicm), 10)
print("Mean spread for dynamic p (DHICM): ", mean_spread)
print("Duration: ", time.time() - start_time)

Mean spread for dynamic p (DHICM):  5074.4
Duration:  304.3230299949646


In [ ]:
start_time = time.time()
#getting mean spread
(mean_spread, A) = modified_ICM(G, list(s_degree), 10)
print("Mean spread for dynamic p (Degree Centrality): ", mean_spread)
print("Duration: ", time.time() - start_time)

Mean spread for dynamic p (Degree Centrality):  5083.5
Duration:  302.0730185508728
